## Functions covered here:
* collect_list
* array_contains
* arrays_zip
* map_from_arrays
* array_distinct
* array_except
* array_intersect
* array_join
* array_remove
* element_at
* array_position
* array_max
* array_repeat
* array_sort
* sort_array
* array_union
* concat
* array
* flatten
* arrays_overlap
* reverse
* shuffle
* size
* slice
* explode
* posexplode
* posexplode_outer

In [0]:
import pyspark.sql.functions as f
initial_df = sc.parallelize([("x", 4, 1),
  ("x", 6, 2),
  ("z", 7, 3),
  ("a", 3, 4),
  ("z", 5, 2),
  ("x", 7, 3),
  ("x", 9, 7),
  ("z", 1, 8),
  ("z", 4, 9),
  ("z", 7, 4),
  ("a", 8, 5),
  ("a", 5, 2),
  ("a", 3, 8),
  ("x", 2, 7),
  ("z", 1, 9)]).toDF(["col1", "col2", "col3"])
initial_df.show()

+----+----+----+
|col1|col2|col3|
+----+----+----+
|   x|   4|   1|
|   x|   6|   2|
|   z|   7|   3|
|   a|   3|   4|
|   z|   5|   2|
|   x|   7|   3|
|   x|   9|   7|
|   z|   1|   8|
|   z|   4|   9|
|   z|   7|   4|
|   a|   8|   5|
|   a|   5|   2|
|   a|   3|   8|
|   x|   2|   7|
|   z|   1|   9|
+----+----+----+



In [0]:
initial_df.dtypes

Out[2]: [('col1', 'string'), ('col2', 'bigint'), ('col3', 'bigint')]

In [0]:
df=initial_df.groupBy("col1").agg(f.collect_list("col2").alias("array_c2"),f.collect_list("col3").alias("array_c3"))
df.show()
df.printSchema()


+----+------------------+------------------+
|col1|          array_c2|          array_c3|
+----+------------------+------------------+
|   x|   [4, 6, 7, 9, 2]|   [1, 2, 3, 7, 7]|
|   z|[7, 5, 1, 4, 7, 1]|[3, 2, 8, 9, 4, 9]|
|   a|      [3, 8, 5, 3]|      [4, 5, 2, 8]|
+----+------------------+------------------+

root
 |-- col1: string (nullable = true)
 |-- array_c2: array (nullable = false)
 |    |-- element: long (containsNull = false)
 |-- array_c3: array (nullable = false)
 |    |-- element: long (containsNull = false)



In [0]:
df.select('col1',f.explode('array_c2')).show()

+----+---+
|col1|col|
+----+---+
|   x|  4|
|   x|  6|
|   x|  7|
|   x|  9|
|   x|  2|
|   z|  7|
|   z|  5|
|   z|  1|
|   z|  4|
|   z|  7|
|   z|  1|
|   a|  3|
|   a|  8|
|   a|  5|
|   a|  3|
+----+---+



In [0]:
df.withColumn("result",f.array_contains("array_c2",3)).show()

+----+------------------+------------------+------+
|col1|          array_c2|          array_c3|result|
+----+------------------+------------------+------+
|   x|   [4, 6, 7, 9, 2]|   [1, 2, 3, 7, 7]| false|
|   z|[7, 5, 1, 4, 7, 1]|[3, 2, 8, 9, 4, 9]| false|
|   a|      [3, 8, 5, 3]|      [4, 5, 2, 8]|  true|
+----+------------------+------------------+------+



In [0]:
df.withColumn("result",f.arrays_zip("array_c2","array_c3")).show(truncate=False)

+----+------------------+------------------+------------------------------------------------+
|col1|array_c2          |array_c3          |result                                          |
+----+------------------+------------------+------------------------------------------------+
|x   |[4, 6, 7, 9, 2]   |[1, 2, 3, 7, 7]   |[{4, 1}, {6, 2}, {7, 3}, {9, 7}, {2, 7}]        |
|z   |[7, 5, 1, 4, 7, 1]|[3, 2, 8, 9, 4, 9]|[{7, 3}, {5, 2}, {1, 8}, {4, 9}, {7, 4}, {1, 9}]|
|a   |[3, 8, 5, 3]      |[4, 5, 2, 8]      |[{3, 4}, {8, 5}, {5, 2}, {3, 8}]                |
+----+------------------+------------------+------------------------------------------------+



In [0]:
df3=spark.createDataFrame([([2, 5,2], ['a', 'b','c'])], schema=['k', 'v'])
df3.show()
df3.withColumn("result",f.map_from_arrays("k","v")).show(truncate=False)

+---------+---------+
|        k|        v|
+---------+---------+
|[2, 5, 2]|[a, b, c]|
+---------+---------+



---------------------------------------------------------------------------
Py4JJavaError                             Traceback (most recent call last)
<command-2456600327339300> in <module>
      1 df3=spark.createDataFrame([([2, 5,2], ['a', 'b','c'])], schema=['k', 'v'])
      2 df3.show()
----> 3 df3.withColumn("result",f.map_from_arrays("k","v")).show(truncate=False)

/databricks/spark/python/pyspark/sql/dataframe.py in show(self, n, truncate, vertical)
    508                     "Parameter 'truncate={}' should be either bool or int.".format(truncate))
    509 
--> 510             print(self._jdf.showString(n, int_truncate, vertical))
    511 
    512     def __repr__(self):

/databricks/spark/python/lib/py4j-0.10.9.1-src.zip/py4j/java_gateway.py in __call__(self, *args)
   1302 
   1303         answer = self.gateway_client.send_command(command)
-> 1304         return_value = get_return_value(
   1305             answer, self.gateway_client, self.target_id, self.name)
   1306 

/d

In [0]:
df3=spark.createDataFrame([([2, 5,7], ['a', 'b','c'])], schema=['k', 'v'])
df3.show()
df3.withColumn("result",f.map_from_arrays("k","v")).show(truncate=False)

+---------+---------+
|        k|        v|
+---------+---------+
|[2, 5, 7]|[a, b, c]|
+---------+---------+

+---------+---------+------------------------+
|k        |v        |result                  |
+---------+---------+------------------------+
|[2, 5, 7]|[a, b, c]|{2 -> a, 5 -> b, 7 -> c}|
+---------+---------+------------------------+



In [0]:
df.withColumn("result",f.array_distinct("array_c2")).show()

+----+------------------+------------------+---------------+
|col1|          array_c2|          array_c3|         result|
+----+------------------+------------------+---------------+
|   x|   [4, 6, 7, 9, 2]|   [1, 2, 3, 7, 7]|[4, 6, 7, 9, 2]|
|   z|[7, 5, 1, 4, 7, 1]|[3, 2, 8, 9, 4, 9]|   [7, 5, 1, 4]|
|   a|      [3, 8, 5, 3]|      [4, 5, 2, 8]|      [3, 8, 5]|
+----+------------------+------------------+---------------+



In [0]:
df.withColumn("result",f.array_except("array_c3","array_c2")).show()

+----+------------------+------------------+------------+
|col1|          array_c2|          array_c3|      result|
+----+------------------+------------------+------------+
|   x|   [4, 6, 7, 9, 2]|   [1, 2, 3, 7, 7]|      [1, 3]|
|   z|[7, 5, 1, 4, 7, 1]|[3, 2, 8, 9, 4, 9]|[3, 2, 8, 9]|
|   a|      [3, 8, 5, 3]|      [4, 5, 2, 8]|      [4, 2]|
+----+------------------+------------------+------------+



In [0]:
df.withColumn("result",f.array_intersect("array_c2","array_c3")).show()

+----+------------------+------------------+------+
|col1|          array_c2|          array_c3|result|
+----+------------------+------------------+------+
|   x|   [4, 6, 7, 9, 2]|   [1, 2, 3, 7, 7]|[7, 2]|
|   z|[7, 5, 1, 4, 7, 1]|[3, 2, 8, 9, 4, 9]|   [4]|
|   a|      [3, 8, 5, 3]|      [4, 5, 2, 8]|[8, 5]|
+----+------------------+------------------+------+



In [0]:
df.withColumn("result",f.array_join("array_c2","-")).show()

+----+------------------+------------------+-----------+
|col1|          array_c2|          array_c3|     result|
+----+------------------+------------------+-----------+
|   x|   [4, 6, 7, 9, 2]|   [1, 2, 3, 7, 7]|  4-6-7-9-2|
|   z|[7, 5, 1, 4, 7, 1]|[3, 2, 8, 9, 4, 9]|7-5-1-4-7-1|
|   a|      [3, 8, 5, 3]|      [4, 5, 2, 8]|    3-8-5-3|
+----+------------------+------------------+-----------+



In [0]:
df.withColumn("result",f.array_max("array_c2")).show()

+----+------------------+------------------+------+
|col1|          array_c2|          array_c3|result|
+----+------------------+------------------+------+
|   x|   [4, 6, 7, 9, 2]|   [1, 2, 3, 7, 7]|     9|
|   z|[7, 5, 1, 4, 7, 1]|[3, 2, 8, 9, 4, 9]|     7|
|   a|      [3, 8, 5, 3]|      [4, 5, 2, 8]|     8|
+----+------------------+------------------+------+



In [0]:
df.withColumn("result",f.array_position("array_c2",7)).show()

+----+------------------+------------------+------+
|col1|          array_c2|          array_c3|result|
+----+------------------+------------------+------+
|   x|   [4, 6, 7, 9, 2]|   [1, 2, 3, 7, 7]|     3|
|   z|[7, 5, 1, 4, 7, 1]|[3, 2, 8, 9, 4, 9]|     1|
|   a|      [3, 8, 5, 3]|      [4, 5, 2, 8]|     0|
+----+------------------+------------------+------+



In [0]:
df.withColumn("result",f.element_at("array_c3",3)).show()

+----+------------------+------------------+------+
|col1|          array_c2|          array_c3|result|
+----+------------------+------------------+------+
|   x|   [4, 6, 7, 9, 2]|   [1, 2, 3, 7, 7]|     3|
|   z|[7, 5, 1, 4, 7, 1]|[3, 2, 8, 9, 4, 9]|     8|
|   a|      [3, 8, 5, 3]|      [4, 5, 2, 8]|     2|
+----+------------------+------------------+------+



In [0]:
df.withColumn("result",f.array_remove("array_c2",7)).show()

+----+------------------+------------------+------------+
|col1|          array_c2|          array_c3|      result|
+----+------------------+------------------+------------+
|   x|   [4, 6, 7, 9, 2]|   [1, 2, 3, 7, 7]|[4, 6, 9, 2]|
|   z|[7, 5, 1, 4, 7, 1]|[3, 2, 8, 9, 4, 9]|[5, 1, 4, 1]|
|   a|      [3, 8, 5, 3]|      [4, 5, 2, 8]|[3, 8, 5, 3]|
+----+------------------+------------------+------------+



In [0]:
df.withColumn("result",f.array_repeat("array_c2",5)).show(truncate=False)

+----+------------------+------------------+----------------------------------------------------------------------------------------------------+
|col1|array_c2          |array_c3          |result                                                                                              |
+----+------------------+------------------+----------------------------------------------------------------------------------------------------+
|x   |[4, 6, 7, 9, 2]   |[1, 2, 3, 7, 7]   |[[4, 6, 7, 9, 2], [4, 6, 7, 9, 2], [4, 6, 7, 9, 2], [4, 6, 7, 9, 2], [4, 6, 7, 9, 2]]               |
|z   |[7, 5, 1, 4, 7, 1]|[3, 2, 8, 9, 4, 9]|[[7, 5, 1, 4, 7, 1], [7, 5, 1, 4, 7, 1], [7, 5, 1, 4, 7, 1], [7, 5, 1, 4, 7, 1], [7, 5, 1, 4, 7, 1]]|
|a   |[3, 8, 5, 3]      |[4, 5, 2, 8]      |[[3, 8, 5, 3], [3, 8, 5, 3], [3, 8, 5, 3], [3, 8, 5, 3], [3, 8, 5, 3]]                              |
+----+------------------+------------------+--------------------------------------------------------------------------------

In [0]:
df.withColumn("result",f.array_sort("array_c2")).show(truncate=False)

+----+------------------+------------------+------------------+
|col1|array_c2          |array_c3          |result            |
+----+------------------+------------------+------------------+
|x   |[4, 6, 7, 9, 2]   |[1, 2, 3, 7, 7]   |[2, 4, 6, 7, 9]   |
|z   |[7, 5, 1, 4, 7, 1]|[3, 2, 8, 9, 4, 9]|[1, 1, 4, 5, 7, 7]|
|a   |[3, 8, 5, 3]      |[4, 5, 2, 8]      |[3, 3, 5, 8]      |
+----+------------------+------------------+------------------+



In [0]:
df.withColumn("result",f.sort_array("array_c2",asc=False)).show(truncate=False)

+----+------------------+------------------+------------------+
|col1|array_c2          |array_c3          |result            |
+----+------------------+------------------+------------------+
|x   |[4, 6, 7, 9, 2]   |[1, 2, 3, 7, 7]   |[9, 7, 6, 4, 2]   |
|z   |[7, 5, 1, 4, 7, 1]|[3, 2, 8, 9, 4, 9]|[7, 7, 5, 4, 1, 1]|
|a   |[3, 8, 5, 3]      |[4, 5, 2, 8]      |[8, 5, 3, 3]      |
+----+------------------+------------------+------------------+



In [0]:
df.withColumn("result",f.array_union("array_c2","array_c3")).show(truncate=False)

+----+------------------+------------------+------------------------+
|col1|array_c2          |array_c3          |result                  |
+----+------------------+------------------+------------------------+
|x   |[4, 6, 7, 9, 2]   |[1, 2, 3, 7, 7]   |[4, 6, 7, 9, 2, 1, 3]   |
|z   |[7, 5, 1, 4, 7, 1]|[3, 2, 8, 9, 4, 9]|[7, 5, 1, 4, 3, 2, 8, 9]|
|a   |[3, 8, 5, 3]      |[4, 5, 2, 8]      |[3, 8, 5, 4, 2]         |
+----+------------------+------------------+------------------------+



In [0]:
df.withColumn("result",f.concat("array_c2","array_c3")).show(truncate=False)

+----+------------------+------------------+------------------------------------+
|col1|array_c2          |array_c3          |result                              |
+----+------------------+------------------+------------------------------------+
|x   |[4, 6, 7, 9, 2]   |[1, 2, 3, 7, 7]   |[4, 6, 7, 9, 2, 1, 2, 3, 7, 7]      |
|z   |[7, 5, 1, 4, 7, 1]|[3, 2, 8, 9, 4, 9]|[7, 5, 1, 4, 7, 1, 3, 2, 8, 9, 4, 9]|
|a   |[3, 8, 5, 3]      |[4, 5, 2, 8]      |[3, 8, 5, 3, 4, 5, 2, 8]            |
+----+------------------+------------------+------------------------------------+



In [0]:
df.withColumn("result",f.array("array_c2","array_c3")).show(truncate=False)

+----+------------------+------------------+----------------------------------------+
|col1|array_c2          |array_c3          |result                                  |
+----+------------------+------------------+----------------------------------------+
|x   |[4, 6, 7, 9, 2]   |[1, 2, 3, 7, 7]   |[[4, 6, 7, 9, 2], [1, 2, 3, 7, 7]]      |
|z   |[7, 5, 1, 4, 7, 1]|[3, 2, 8, 9, 4, 9]|[[7, 5, 1, 4, 7, 1], [3, 2, 8, 9, 4, 9]]|
|a   |[3, 8, 5, 3]      |[4, 5, 2, 8]      |[[3, 8, 5, 3], [4, 5, 2, 8]]            |
+----+------------------+------------------+----------------------------------------+



In [0]:
df2=df.withColumn("result",f.array("array_c2","array_c3"))
df2.withColumn("result1",f.flatten("result")).show(truncate=False)

+----+------------------+------------------+----------------------------------------+------------------------------------+
|col1|array_c2          |array_c3          |result                                  |result1                             |
+----+------------------+------------------+----------------------------------------+------------------------------------+
|x   |[4, 6, 7, 9, 2]   |[1, 2, 3, 7, 7]   |[[4, 6, 7, 9, 2], [1, 2, 3, 7, 7]]      |[4, 6, 7, 9, 2, 1, 2, 3, 7, 7]      |
|z   |[7, 5, 1, 4, 7, 1]|[3, 2, 8, 9, 4, 9]|[[7, 5, 1, 4, 7, 1], [3, 2, 8, 9, 4, 9]]|[7, 5, 1, 4, 7, 1, 3, 2, 8, 9, 4, 9]|
|a   |[3, 8, 5, 3]      |[4, 5, 2, 8]      |[[3, 8, 5, 3], [4, 5, 2, 8]]            |[3, 8, 5, 3, 4, 5, 2, 8]            |
+----+------------------+------------------+----------------------------------------+------------------------------------+



In [0]:
df.withColumn("result",f.arrays_overlap("array_c2","array_c3")).show(truncate=False)

+----+------------------+------------------+------+
|col1|array_c2          |array_c3          |result|
+----+------------------+------------------+------+
|x   |[4, 6, 7, 9, 2]   |[1, 2, 3, 7, 7]   |true  |
|z   |[7, 5, 1, 4, 7, 1]|[3, 2, 8, 9, 4, 9]|true  |
|a   |[3, 8, 5, 3]      |[4, 5, 2, 8]      |true  |
+----+------------------+------------------+------+



In [0]:
df.withColumn("result",f.reverse("array_c2")).show()

+----+------------------+------------------+------------------+
|col1|          array_c2|          array_c3|            result|
+----+------------------+------------------+------------------+
|   x|   [4, 6, 7, 9, 2]|   [1, 2, 3, 7, 7]|   [2, 9, 7, 6, 4]|
|   z|[7, 5, 1, 4, 7, 1]|[3, 2, 8, 9, 4, 9]|[1, 7, 4, 1, 5, 7]|
|   a|      [3, 8, 5, 3]|      [4, 5, 2, 8]|      [3, 5, 8, 3]|
+----+------------------+------------------+------------------+



In [0]:
df.withColumn("result",f.shuffle("array_c2")).show()

+----+------------------+------------------+------------------+
|col1|          array_c2|          array_c3|            result|
+----+------------------+------------------+------------------+
|   x|   [4, 6, 7, 9, 2]|   [1, 2, 3, 7, 7]|   [9, 4, 2, 6, 7]|
|   z|[7, 5, 1, 4, 7, 1]|[3, 2, 8, 9, 4, 9]|[1, 5, 4, 7, 7, 1]|
|   a|      [3, 8, 5, 3]|      [4, 5, 2, 8]|      [8, 5, 3, 3]|
+----+------------------+------------------+------------------+



In [0]:
df.withColumn("result",f.size("array_c2")).show()

+----+------------------+------------------+------+
|col1|          array_c2|          array_c3|result|
+----+------------------+------------------+------+
|   x|   [4, 6, 7, 9, 2]|   [1, 2, 3, 7, 7]|     5|
|   z|[7, 5, 1, 4, 7, 1]|[3, 2, 8, 9, 4, 9]|     6|
|   a|      [3, 8, 5, 3]|      [4, 5, 2, 8]|     4|
+----+------------------+------------------+------+



In [0]:
df.withColumn("result",f.slice("array_c2",2,3)).show()

+----+------------------+------------------+---------+
|col1|          array_c2|          array_c3|   result|
+----+------------------+------------------+---------+
|   x|   [4, 6, 7, 9, 2]|   [1, 2, 3, 7, 7]|[6, 7, 9]|
|   z|[7, 5, 1, 4, 7, 1]|[3, 2, 8, 9, 4, 9]|[5, 1, 4]|
|   a|      [3, 8, 5, 3]|      [4, 5, 2, 8]|[8, 5, 3]|
+----+------------------+------------------+---------+



In [0]:
df.withColumn("result",f.explode("array_c2")).show()

+----+------------------+------------------+------+
|col1|          array_c2|          array_c3|result|
+----+------------------+------------------+------+
|   x|   [4, 6, 7, 9, 2]|   [1, 2, 3, 7, 7]|     4|
|   x|   [4, 6, 7, 9, 2]|   [1, 2, 3, 7, 7]|     6|
|   x|   [4, 6, 7, 9, 2]|   [1, 2, 3, 7, 7]|     7|
|   x|   [4, 6, 7, 9, 2]|   [1, 2, 3, 7, 7]|     9|
|   x|   [4, 6, 7, 9, 2]|   [1, 2, 3, 7, 7]|     2|
|   z|[7, 5, 1, 4, 7, 1]|[3, 2, 8, 9, 4, 9]|     7|
|   z|[7, 5, 1, 4, 7, 1]|[3, 2, 8, 9, 4, 9]|     5|
|   z|[7, 5, 1, 4, 7, 1]|[3, 2, 8, 9, 4, 9]|     1|
|   z|[7, 5, 1, 4, 7, 1]|[3, 2, 8, 9, 4, 9]|     4|
|   z|[7, 5, 1, 4, 7, 1]|[3, 2, 8, 9, 4, 9]|     7|
|   z|[7, 5, 1, 4, 7, 1]|[3, 2, 8, 9, 4, 9]|     1|
|   a|      [3, 8, 5, 3]|      [4, 5, 2, 8]|     3|
|   a|      [3, 8, 5, 3]|      [4, 5, 2, 8]|     8|
|   a|      [3, 8, 5, 3]|      [4, 5, 2, 8]|     5|
|   a|      [3, 8, 5, 3]|      [4, 5, 2, 8]|     3|
+----+------------------+------------------+------+



In [0]:
df.select("*",f.posexplode("array_c2")).show()

+----+------------------+------------------+---+---+
|col1|          array_c2|          array_c3|pos|col|
+----+------------------+------------------+---+---+
|   x|   [4, 6, 7, 9, 2]|   [1, 2, 3, 7, 7]|  0|  4|
|   x|   [4, 6, 7, 9, 2]|   [1, 2, 3, 7, 7]|  1|  6|
|   x|   [4, 6, 7, 9, 2]|   [1, 2, 3, 7, 7]|  2|  7|
|   x|   [4, 6, 7, 9, 2]|   [1, 2, 3, 7, 7]|  3|  9|
|   x|   [4, 6, 7, 9, 2]|   [1, 2, 3, 7, 7]|  4|  2|
|   z|[7, 5, 1, 4, 7, 1]|[3, 2, 8, 9, 4, 9]|  0|  7|
|   z|[7, 5, 1, 4, 7, 1]|[3, 2, 8, 9, 4, 9]|  1|  5|
|   z|[7, 5, 1, 4, 7, 1]|[3, 2, 8, 9, 4, 9]|  2|  1|
|   z|[7, 5, 1, 4, 7, 1]|[3, 2, 8, 9, 4, 9]|  3|  4|
|   z|[7, 5, 1, 4, 7, 1]|[3, 2, 8, 9, 4, 9]|  4|  7|
|   z|[7, 5, 1, 4, 7, 1]|[3, 2, 8, 9, 4, 9]|  5|  1|
|   a|      [3, 8, 5, 3]|      [4, 5, 2, 8]|  0|  3|
|   a|      [3, 8, 5, 3]|      [4, 5, 2, 8]|  1|  8|
|   a|      [3, 8, 5, 3]|      [4, 5, 2, 8]|  2|  5|
|   a|      [3, 8, 5, 3]|      [4, 5, 2, 8]|  3|  3|
+----+------------------+------------------+--

In [0]:
#Unlike posexplode, if the array/map is null or empty then the row (null, null) is produced.
df.select("*",f.posexplode_outer("array_c2")).show()

+----+------------------+------------------+---+---+
|col1|          array_c2|          array_c3|pos|col|
+----+------------------+------------------+---+---+
|   x|   [4, 6, 7, 9, 2]|   [1, 2, 3, 7, 7]|  0|  4|
|   x|   [4, 6, 7, 9, 2]|   [1, 2, 3, 7, 7]|  1|  6|
|   x|   [4, 6, 7, 9, 2]|   [1, 2, 3, 7, 7]|  2|  7|
|   x|   [4, 6, 7, 9, 2]|   [1, 2, 3, 7, 7]|  3|  9|
|   x|   [4, 6, 7, 9, 2]|   [1, 2, 3, 7, 7]|  4|  2|
|   z|[7, 5, 1, 4, 7, 1]|[3, 2, 8, 9, 4, 9]|  0|  7|
|   z|[7, 5, 1, 4, 7, 1]|[3, 2, 8, 9, 4, 9]|  1|  5|
|   z|[7, 5, 1, 4, 7, 1]|[3, 2, 8, 9, 4, 9]|  2|  1|
|   z|[7, 5, 1, 4, 7, 1]|[3, 2, 8, 9, 4, 9]|  3|  4|
|   z|[7, 5, 1, 4, 7, 1]|[3, 2, 8, 9, 4, 9]|  4|  7|
|   z|[7, 5, 1, 4, 7, 1]|[3, 2, 8, 9, 4, 9]|  5|  1|
|   a|      [3, 8, 5, 3]|      [4, 5, 2, 8]|  0|  3|
|   a|      [3, 8, 5, 3]|      [4, 5, 2, 8]|  1|  8|
|   a|      [3, 8, 5, 3]|      [4, 5, 2, 8]|  2|  5|
|   a|      [3, 8, 5, 3]|      [4, 5, 2, 8]|  3|  3|
+----+------------------+------------------+--